In [1]:
'''
This is the main tool. I have tried defining it into
its own function, but loading model checkpoints does 
not seem to allow that. Nonetheless, it can be easily
controlled from here

This extracts the inner layers of a trained model, 
first by loading saved checkpoint of model
then running a forward pass through the model

As soon as this is placed inside a function,
the ability to load the model is lost for some reason.

'''

import torch
import schnetpack as spk
import math
from schnetpack.datasets import QM9
import os

import numpy as np   
from numpy import savetxt

import pandas as pd

element = 'F'

#inputs
save_path = '../../data/schnet/%s/' %(element) 
main_name = 'rep'
# convention dataset.energy.#training-#filters
model_name = 'qm9energy10000-30'

number_inputs=5000
sub_name = ''
qm9_file = '../../data/datasets/QM9/qm9.db'
#label_file =  '../../data/labeldataset/%s/label%s%s.csv' %(element,element,number_inputs)




In [2]:
qm9data = QM9(qm9_file, download=False, remove_uncharacterized=True)



name_data = main_name + model_name + element + str(number_inputs) + sub_name

#read in label file in order to output specific functional groups
#label = pd.read_csv(label_file,delimiter=',')

split_file='../../data/trainedmodels/%s/split.npz' %(model_name)
checkpoint_path = '../../data/trainedmodels/%s/trained.pth' %(model_name)
n_atom_basis=30
n_filters=30
n_gaussians=20
n_interactions=3
cutoff = 4.



In [3]:
def hook(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global layer
    #Update the embedding_output variable to be equal to our output tensor
    layer=out_tensor 

# Full Representation X = X0 + V1 + V2 + V3

In [4]:
# Load split file 
train, val, test = spk.data.train_test_split(qm9data,split_file=split_file)

# Load atom ref data 
atomrefs = qm9data.get_atomref(QM9.U0)

# Define SchNet representation model

schnet = spk.representation.SchNet(
n_atom_basis=n_atom_basis, n_filters=n_filters, n_gaussians=n_gaussians, n_interactions=n_interactions,
cutoff=cutoff , cutoff_network=spk.nn.cutoff.CosineCutoff
)

# Define SchNet output model and property to be predicted

output_U0 = spk.atomistic.Atomwise(n_in=n_filters, atomref=atomrefs[QM9.U0])

# Define atomistic model

model = spk.AtomisticModel(representation=schnet,output_modules=output_U0)

# Load saved checkpoint file
load_checkpoint = torch.load(checkpoint_path)


#qm9_i6_30f_20g-1000-500-4_300.pth
# load model's state dictionary from saved checkpoint
model.load_state_dict(load_checkpoint)


#set up device for forward pass
device='cpu'

# load atoms converter 
converter = spk.data.AtomsConverter(device=device)

data = np.zeros((1,31))
dataae = np.zeros((1,1))
countO = 0
for idx in range(number_inputs):
    at, props = qm9data.get_properties(idx)
    inputs = converter(at)
    
    layer = None
    
    model.representation.embedding.register_forward_hook(hook)
    model(inputs)
    
    emb = layer.clone()
    emb = layer.detach().numpy()
    
    layer = None
    
    model.representation.interactions[0].register_forward_hook(hook)
    
    model(inputs)
    int0 = layer.clone()
    int0 = int0.detach().numpy()    

    layer = None
    
    model.representation.interactions[1].register_forward_hook(hook)
    
    model(inputs)
    int1 = layer.clone()
    int1 = int1.detach().numpy()   
    
    layer = None 
    
    model.representation.interactions[2].register_forward_hook(hook)
    
    model(inputs)   
    
    int2 = layer.clone()
    int2 = int2.detach().numpy()   

    rep = emb+int0+int1+int2
    
    number_atoms = len(props['_positions'])
    

    from schnetpack.atomistic.output_modules import yi
    
    rows = np.zeros((number_atoms,31))
    for i in range(number_atoms):
        for j in range(30):
            rows[i][j] = rep[0][i][j]

    
    yi=yi.detach().numpy()
    #save the vector of every oxygen atom encountered
    
    
#    print(idx)
    for i in range(number_atoms):
#        if props['_atomic_numbers'][i] == 8:
#            if label['Target'][countO] == 2:
#                print(idx)
#                datao = np.vstack((datao,rows[i]))
#                dataoae = np.vstack((dataoae,yi[0][i])) 
#            countO = countO + 1

        if element == 'H': 
            if props['_atomic_numbers'][i] == 1:
                data = np.vstack((data,rows[i])) 
                dataae = np.vstack((dataae,yi[0][i]))
        if element == 'O':
            if props['_atomic_numbers'][i] == 8:
                data = np.vstack((data,rows[i])) 
                dataae = np.vstack((dataae,yi[0][i]))
        if element == 'N':
            if props['_atomic_numbers'][i] == 7:
                data = np.vstack((data,rows[i]))    
                dataae = np.vstack((dataae,yi[0][i]))   
        if element == 'C':
            if props['_atomic_numbers'][i] == 6:
                data = np.vstack((data,rows[i]))    
                dataae = np.vstack((dataae,yi[0][i]))  
        if element == 'F':
            if props['_atomic_numbers'][i] == 9:
                data = np.vstack((data,rows[i]))    
                dataae = np.vstack((dataae,yi[0][i]))  
if not os.path.exists(save_path):
    os.makedirs(save_path)

data = np.delete(data, 0 ,axis=0)
dataae = np.delete(dataae, 0 ,axis=0)

C:\Users\aelsamma\Anaconda3\lib\site-packages\schnetpack\data\atoms.py:327: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  properties[pname] = torch.FloatTensor(prop)
C:\Users\aelsamma\Anaconda3\lib\site-packages\ase\atoms.py:967: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  warnings.warn('Use get_global_number_of_atoms() instead',


In [5]:

savetxt(save_path+name_data+'.csv',data,delimiter=',')

savetxt(save_path+name_data+'ae.csv',dataae,delimiter=',')
